In [1]:
from socket import *
import re
import threading
import os
import mimetypes
import datetime

MASTER_FILE_LIST = ["/index.html", "/sebern1.jpg", "/style.css", "/e-sebern2.gif", "/", "/courses.html"]

def http_server_setup(port):
    """
    Start the HTTP server
    - Open the listening socket
    - Accept connections and spawn processes to handle requests

    :param port: listening port number
    """

    num_connections = 10
    server_socket = socket(AF_INET, SOCK_STREAM)
    listen_address = ('', port)
    server_socket.bind(listen_address)
    server_socket.listen(num_connections)
    try:
        while True:
            request_socket, request_address = server_socket.accept()
            print('connection from {0} {1}'.format(request_address[0],
                                                   request_address[1]))
            # Create a new thread, and set up the handle_request method and its argument (in a tuple)
            request_handler = threading.Thread(target=handle_request,
                                               args=(request_socket, ))
            # Start the request handler thread.
            request_handler.start()
            # Just for information, display the running threads (including this main one)
            print('threads: ', threading.enumerate())
    # Set up so a Ctrl-C should terminate the server; this may have some problems on Windows
    except KeyboardInterrupt:
        print("HTTP server exiting . . .")
        print('threads: ', threading.enumerate())
        server_socket.close()

In [2]:
# You may use these functions to simplify your code.
def get_mime_type(file_path):
    """
    Try to guess the MIME type of a file (resource), given its path (primarily its file extension)

    :param file_path: string containing path to (resource) file, such as './abc.html'
    :return: If successful in guessing the MIME type, a string representing the content type, such as 'text/html'
             Otherwise, None
    :rtype: int or None
    """

    mime_type_and_encoding = mimetypes.guess_type(file_path)
    mime_type = mime_type_and_encoding[0]
    return mime_type


def get_file_size(file_path):
    """
    Try to get the size of a file (resource) as number of bytes, given its path

    :param file_path: string containing path to (resource) file, such as './abc.html'
    :return: If file_path designates a normal file, an integer value representing the the file size in bytes
             Otherwise (no such file, or path is not a file), None
    :rtype: int or None
    """

    # Initially, assume file does not exist
    file_size = None
    if os.path.isfile(file_path):
        file_size = os.stat(file_path).st_size
    return file_size

In [3]:
def build_response(request_header):
    mime = ''
    resource = request_header[1]
    if resource in MASTER_FILE_LIST:
        mime = mimetypes.MimeTypes().guess_type(resource)[0]    
    header = 'HTTP/1.1 200 OK \r\n'
    server = 'Server: Kevins server\r\n'
    contentType = 'Content-Type: ' + mime + '\r\n'
    connection = 'Connection: keep-alive\r\n'
    dt = datetime.datetime.now()
    date = dt.strftime('Date: %d/%m/%Y %H:%M:%S\r\n')
    filePath = '.\\' + resource
    lenFile =  os.path.getsize(filePath)
    content_len = 'Content length: ' + str(lenFile) + '\r\n'
    return header + server + date + contentType + content_len + connection 

In [4]:
def file_in_bytes(request_header):
    cookie = b''
    with open('.\\' + request_header[1], "rb") as f:
        while True:
            # read the bytes from the file
            bytes_read = f.read(1024)
            if not bytes_read:
                # file transmitting is done
                break
            cookie += bytes_read
    return cookie     

In [5]:
def handle_request(request_socket):
    # Get the request line
    message = ''
    while True:
        data = request_socket.recv(1)
        message += data.decode('utf-8')
        if '\r\n' in message:
            break
    request_header = message.split(' ')
    for x in range(len(request_header)):
        print(request_header[x])
    # Get the remaining header lines
    while True:    
        data = request_socket.recv(1)
        message += data.decode('utf-8')
        if '\r\n\r\n' in message:
            break
    #Breaks headers into key value pairs
    allHeaders = dict(re.findall(r'(?=\S|^)(.+?): (\S+)', message))
    #Send back resource requested
    if request_header[1] == '/':
        request_header[1] = 'index.html'
        headers = build_response(request_header)
        body = file_in_bytes(request_header)
        wholeMessage = bytes(headers, 'utf-8') + b'\r\n' + body + b'\r\n\r\n'
        request_socket.sendall(wholeMessage)    
    elif request_header[1] in MASTER_FILE_LIST:
        headers = build_response(request_header)
        body = file_in_bytes(request_header)
        wholeMessage = bytes(headers, 'utf-8')+ b'\r\n' + body + b'\r\n\r\n'
        request_socket.sendall(wholeMessage)     
    else:
        request_socket.sendall(bytes('HTTP/1.1 404 NOT FOUND', 'utf-8'))
    request_socket.close()

In [ ]:
# Start the server
http_server_setup(8080)
# Now  navigate to localhost:8080 in your browser

connection from 127.0.0.1 63932
threads:  [<_MainThread(MainThread, started 59776)>, <Thread(Thread-4, started daemon 56364)>, <Heartbeat(Thread-5, started daemon 51252)>, <HistorySavingThread(IPythonHistorySavingThread, started 58292)>, <ParentPollerWindows(Thread-3, started daemon 50604)>, <Thread(Thread-6, started 56292)>]
connection from 127.0.0.1 64653
GET
/
HTTP/1.1

threads:  [<_MainThread(MainThread, started 59776)>, <Thread(Thread-4, started daemon 56364)>, <Heartbeat(Thread-5, started daemon 51252)>, <HistorySavingThread(IPythonHistorySavingThread, started 58292)>, <ParentPollerWindows(Thread-3, started daemon 50604)>, <Thread(Thread-6, started 56292)>, <Thread(Thread-7, started 63376)>]
GET
/style.css
HTTP/1.1

connection from 127.0.0.1 59696
threads:  [<_MainThread(MainThread, started 59776)>, <Thread(Thread-4, started daemon 56364)>, <Heartbeat(Thread-5, started daemon 51252)>, <HistorySavingThread(IPythonHistorySavingThread, started 58292)>, <ParentPollerWindows(Thread-3,